# An introduction to Pandas

------------------------------------------------------
Machine Learning Year 2019/2020

*Pablo M. Olmos olmos@tsc.uc3m.es*  and *Vanessa Gómez Verdejo vanessa@tsc.uc3m.es*

------------------------------------------------------

When dealing with numeric matrices and vectors in Python, Numerical Python ([Numpy](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html NumPy)) makes life a lot easier. Doing data analysis directly with NumPy can be problematic, as many different data types have to jointly managed.

Fortunately, some nice folks have written the **[Python Data Analysis Library](https://pandas.pydata.org/)** (a.k.a. pandas). Pandas is an open sourcelibrary providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language

In this tutorial, we'll go through the basics of pandas using a database of house prices provided by [Kaggle](https://www.kaggle.com/). Pandas has a lot of functionality, so we'll only be able to cover a small fraction of what you can do. Check out the (very readable) [pandas docs](http://pandas.pydata.org/pandas-docs/stable/) if you want to learn more.

### Acknowledgment:

I have compiled this tutorial by putting together a few very nice blogs and posts I found on the web. All credit goes to them:
- [An introduction to Pandas](http://synesthesiam.com/posts/an-introduction-to-pandas.html#handing-missing-values)
- [Using iloc, loc, & ix to select rows and columns in Pandas DataFrames](https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/)


## Getting Started

Let's import the libray and check the current installed version

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
#The following is required to print the plots inside the notebooks
%matplotlib inline 

In [0]:
pd.__version__

If you are using Anaconda and you want to update pandas to the latest version, you can use either the [package manager](https://docs.anaconda.com/anaconda/navigator/tutorials/manage-packages) in Anaconda Navigator, or type in a terminal window
```
> conda update pandas
```


Next lets read the housing price database, which is described [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data). Because it's in a CSV file, we can use pandas' `read_csv` function to pull it directly into the basic data structure in pandas: a **DataFrame**.

You need to place this csv file in your local working directory ( where you have this notebook). In case you are using Google Colab, go to left pannel (click on left arrow) and select Files->UPLOAD (upload the file to the default folder).

In [0]:
data = pd.read_csv("house_prices_train.csv")

We can visualize the first rows of the Dataframe `data`

In [0]:
data.head()

You have a description of all fields in the [data description file](./data_description.txt).


You can check the size of the Dataframe and get a list of the column labels as follows:

In [0]:
print("The dataframe has %d entries, and %d attributes (columns)\n" %(data.shape[0],data.shape[1]))

print("The labels associated to each of the %d attributes are:\n " %(data.shape[1]))
label_list = list(data.columns)

print(label_list)

Columns can be accessed in two ways. The first is using the DataFrame like a dictionary with string keys:

In [0]:
data[['SalePrice']].head(10)   #This shows the first 10 entries in the column 'SalePrice'

You can get multiple columns out at the same time by passing in a list of strings.

In [0]:
simple_data = data[['LotArea','1stFlrSF','2ndFlrSF','SalePrice']]   
#Subpart of the dataframe. 
# Watch out! This is not a different copy!

simple_data.tail(10) #.tail() shows the last 10 entries

## Operations with columns

We can easily [change the name](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html) of the columns

In [0]:
data.rename(index=str,columns={"LotArea":"Area"}, inplace=True)

Try to rename the column name directly in `simple.data`, what do you get?


There are a lot of useful methods that can be applied over columns. Most of pandas' methods will happily ignore missing values like `NaN`. We will talk about **missing data** later.

First, since we rename one column name, lets recompute the short (referenced) data-frame `simple_data``


In [0]:
simple_data = data[['Area','1stFlrSF','2ndFlrSF','SalePrice']]   
print(simple_data.head(5))

print(simple_data['Area'].mean())
print(simple_data['Area'].std())

Some methods, like plot() and hist() produce plots using [matplotlib](https://matplotlib.org/). We'll go over plotting in more detail later.

In [0]:
simple_data[['Area']][:100].plot()

In [0]:
simple_data[['Area']].hist()

## Operations with `apply()`

Methods like `sum()` and `std()` work on entire columns. We can run our own functions across all values in a column (or row) using `apply()`.

To get an idea about how this works, assume we want to convert the variable ['Area'] into squared meters instead of square foots. First, we create a conversion function.

In [0]:
def sfoot_to_smeter(x):
    "Funtion to convert square foots into squared meters"
    return (x * 0.092903)

In [0]:
sfoot_to_smeter(1) #just checking everything is correct

Using the `apply()` method, which takes an [anonymous function](https://docs.python.org/2/reference/expressions.html#lambda), we can apply `sfoot_to_smeter` to each value in the column. We can now either overwrite the data in the column 'Area' or create a new one. We'll do the latter in this case.

In [0]:
# Recall! data['Area'] is not a DataFrame, but a Pandas Series (another data object with different attributes). In order
# to index a DataFrame with a single column, you should use double [[]], i.e., data[['Area']]

data['Area_m2'] = data[['Area']].apply(lambda d: sfoot_to_smeter(d))

In [0]:
simple_data = data[['Area','Area_m2', '1stFlrSF','2ndFlrSF','SalePrice']]     

simple_data.head()

What do you get if you try to apply the transformation directly over `simple_data`? What do you think the problem is?

Now, we do not even need the column `Area`(in square foot), lets remove it.

In [0]:
data.drop('Area',axis=1,inplace=True)

data.head(5) 

# Indexing, iloc, loc

There are [multiple ways](http://pandas.pydata.org/pandas-docs/stable/indexing.html#different-choices-for-indexing) to select and index rows and columns from Pandas DataFrames. 

There’s three main options to achieve the selection and indexing activities in Pandas, which can be confusing. The three selection cases and methods covered in this post are:

- Selecting data by row numbers (.iloc)
- Selecting data by label or by a conditional statment (.loc)
- Selecting in a hybrid approach (.ix) (now Deprecated in Pandas 0.20.1)

We will cover the first two

### Selecting rows using `iloc()`

The [`iloc`](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.iloc.html) indexer for Pandas Dataframe is used for integer-location based indexing / selection by position.

The iloc indexer syntax is `data.iloc[<row selection>, <column selection>]`. “iloc” in pandas is used to select rows and columns by number, **in the order that they appear in the data frame**. You can imagine that each row has a row number from 0 to the total rows (data.shape[0])  and iloc[] allows selections based on these numbers. The same applies for columns (ranging from 0 to data.shape[1] )

In [0]:
simple_data.iloc[[3,4],0:3]

Note that `.iloc` returns a Pandas Series when one row is selected, and a Pandas DataFrame when multiple rows are selected, or if any column in full is selected. To counter this, pass a single-valued list if you require DataFrame output. 

In [0]:
print(type(simple_data.iloc[:,0]))  #PandaSeries


print(type(simple_data.iloc[:,[0]])) #DataFrame

# To avoid confusion, work always with DataFrames!

When selecting multiple columns or multiple rows in this manner, remember that in your selection e.g.[1:5], the rows/columns selected will run from the first number to one minus the second number. e.g. [1:5] will go 1,2,3,4., [x,y] goes from x to y-1.


In practice, `iloc()` is sheldom used. 'loc()' is way more handly.

### Selecting rows using `loc()`

The Pandas `loc()` indexer can be used with DataFrames for two different use cases:

- Selecting rows by label/index
- Selecting rows with a boolean / conditional lookup

#### Selecting rows by label/index

*Important* Selections using the `loc()` method are based on the index of the data frame (if any). Where the index is set on a DataFrame, using <code>df.set_index()</code>, the `loc()` method directly selects based on index values of any rows. For example, setting the index of our test data frame to the column 'OverallQual' (Rates the overall material and finish of the house):

In [0]:
data.set_index('OverallQual',inplace=True)

In [0]:
data.head(5) 

Using `.loc()` we can search for rows with a specific index value

In [0]:
good_houses = data.loc[[8,9,10]]  #List all houses with rating above 8

good_houses.head(10)

We can sort the dataframe according to index

In [0]:
data.sort_index(inplace=True,ascending=False) #Again, what is what you get if soft Dataframe good_houses directly? 
good_houses.head(10)

#### Boolean / Logical indexing using .loc

[Conditional selections](http://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing) with boolean arrays using `data.loc[<selection>]` is a common method with Pandas DataFrames. With boolean indexing or logical selection, you pass an array or Series of `True/False` values to the `.loc` indexer to select the rows where your Series has True values.

For example, the statement data[‘first_name’] == ‘Antonio’] produces a Pandas Series with a True/False value for every row in the ‘data’ DataFrame, where there are “True” values for the rows where the first_name is “Antonio”. These type of boolean arrays can be passed directly to the .loc indexer as so:

In [0]:
good_houses.loc[good_houses['PoolArea']>0] #How many houses with quality above or equal to 8 have a Pool

As before, a second argument can be passed to .loc to select particular columns out of the data frame.

In [0]:
good_houses.loc[good_houses['PoolArea']>0,['GarageArea','GarageCars']] #Among those above, we focus on the area of the
# garage and how many cars can fit within

Even an anonymous function with the `.apply()` method can be used to generate the series of True/False indexes. For instance, select good houses with less than 10 years.

In [0]:
def check_date(current_year,year_built,threshold):
    
    return (current_year-year_built) <= threshold

In [0]:
good_houses.loc[good_houses['YearBuilt'].apply(lambda d: check_date(2018, d,10))]

Using the above filtering, we can add our own column to the DataFrame to create an index that is 1 for houses that have swimming pool and less than 30 years.

In [0]:
data['My_index'] = 0 # We create new column with default vale

data.loc[(data['YearBuilt'].apply(lambda d: check_date(2018, d,30))) & (data['PoolArea']>0),'My_index'] = 1

data.loc[data['My_index'] == 1]

## Handling Missing Data

Pandas considers values like `NaN` and `None` to represent missing data. The `pandas.isnull` function can be used to tell whether or not a value is missing.

Let's use `apply()` across all of the columns in our DataFrame to figure out which values are missing.

In [0]:
empty = data.apply(lambda col: pd.isnull(col))

empty.head(5) #We get back a boolean Dataframe with 'True' whenever we have a missing data (either Nan or None)

There are multiple ways of handling missing data, we will talk about this during the course. Pandas provides handly functions to easily work with missing data, check [this post](https://chrisalbon.com/python/data_wrangling/pandas_missing_data/) for examples.

## More about plotting with `matplotlib()` library


You should consult [matplotlib documentation](https://matplotlib.org/index.html) for tons of examples and options.

In [0]:
plt.plot(data['Area_m2'],data['SalePrice'],'ro')
plt.plot(good_houses['Area_m2'],good_houses['SalePrice'],'*')
plt.legend(['SalePrice (all data)','SalePrince (good houses)'])
plt.xlabel('Area_m2')
plt.grid(True)
plt.xlim([0,7500])

In [0]:
data.sort_values(['SalePrice'],ascending=True,inplace=True) #We order the data according to SalePrice

# Create axes
fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.loglog(data['SalePrice'], data['Area_m2'], color='blue',marker='o')
ax.set_xlabel('SalePrice (logscale)')
ax.set_ylabel('Area_m2 (logscale)')
ax2.semilogx(data['SalePrice'],data[['GarageArea']].apply(lambda d: sfoot_to_smeter(d)), color='red',marker='+',linewidth=0)
ax2.set_ylabel('Garage Area (logscale)')

ax.set_title('A plot with two scales')

## Getting data out

Writing data out in pandas is as easy as getting data in. To save our DataFrame out to a new csv file, we can just do this:

In [0]:
data.to_csv("modified_data.csv")

There's also support for reading and writing [Excel files](http://pandas.pydata.org/pandas-docs/stable/io.html#excel-files), if you need it.

Also, creating a Numpy array is straightforward:

In [0]:
import numpy as np

In [0]:
data_array = np.array(good_houses)

print(data_array.shape)

## Some exercises

1. We will work with a reduced database, create a **copy** of the original Dataframe called `rdata` (not a reference!, use the [`pd.DataFrame.copy()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.copy.html) method) that only contains the columns

- Area (in square meters!)
- GarageArea (in square meters!)
- YrSold 
- MoSold
- CentralAir
- YearBuilt
- OverallCond
- SalePrice

Check that you are working with a copy and not with a reference of the original dataframe. To do so, check this by renaming a column of the new dataframe.

In [0]:
#Your code here

2. Using the `.apply()` method, create a new column corresponding to the age of the house in months.

In [0]:
#Your code here

3. It is crucial to understand and visualize each attribute. Using a foor loop that iterates over the columns of `reduced_database`, plot both the histogram of each column with 10 bins and the value of each entry with respect to 'SalePrice'. To do so, use [`plt.subplots()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html) to create a figure with two plots. You have examples [here](https://matplotlib.org/gallery/subplots_axes_and_figures/subplots_demo.html).

In [0]:
#Complete the following code
   
for c in rdata: #check what c is!
    
    f, ax = plt.subplots(2,1,figsize=(8, 8))
    ax[0].hist(XX)     #Your code here!
    ax[1].loglog(rdata['SalePrice'],XX,'*') #Your code here!
    ax[0].set_xlabel('SalePrice')
    ax[1].set_xlabel('SalePrice')
    ax[0].set_title('Column: ' + c )

3. Compute the mean and variance of each column in `rdata`. Repeat by considering only those houses with central air conditioning, which entry seems to be more correlated with the fact that a house has central air conditioning?

In [0]:
#Your code here

4. Replace missing data in each column by a representative value. For instance, it can be the mean in real-valued variables, or the most common category in categorical variables (like central air conditioning).

In [0]:
#Your code here

5. Save the results in a cvs file named `Reduced_database_with_missing_inputation.csv``

In [0]:
#Your code here